## ETL

In [220]:
# Librerias necesarias
import pandas as pd
import numpy as np
import json
import ast
import gzip

In [221]:
# Ruta de los datos:
ruta_games= 'Datasets originales/steam_games.json.gz'
ruta_user_reviews= 'Datasets originales/user_reviews.json.gz'
ruta_user_items= 'Datasets originales/users_items.json.gz'

#### Games

In [222]:
## Games:
games_orig = []
with gzip.open(ruta_games, 'rt', encoding='utf-8') as archivo_comprimido:
    for linea in archivo_comprimido:
        diccionario = json.loads(linea)
        games_orig.append(diccionario)

# Transformamos a DataFrame:
games_orig = pd.DataFrame(games_orig)

In [223]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
games = games_orig.copy()
games.shape

(120445, 13)

In [ ]:
games.tail()

Las transformaciones que realizaremos en este caso serán: 

1. Eliminar registros completamente vacíos. 
2. Eliminar columnas que no utilizaremos: ``publisher``, ``url``, ``reviews_url``, ``price``, ``early_access``, ``developer``. Eliminar la columna ``tags`` previo a rellenar los datos faltantes de genres con sus valores. ``app_name`` y ``title`` dicen lo mismo, dejaremos la columna que tenga menor cantidad de datos faltantes.
3. Desanidar registros que poseen valores con tipo de dato LISTA.
4. Eliminar registros duplicados.
5. Corregir el tipo de dato de cada columna. Los registros que poseen el campo ``release_date`` vacío o con un dato que no corresponde, se rellenaran con '1900-01-01' para poder mantener el registro sin perjudicar el análisis. También creamos ``release_year``.
6. Eliminar registros que poseen datos vacíos en columnas relevantes.
7. Renombraremos la columna ``id`` a ``ìtem_id``.

In [224]:
# 1. Eliminar registros completamente vacíos
print(f'tamaño inicial del dataframe: {len(games)} registros')
games.dropna(how='all', inplace= True, ignore_index=True)
print(f'tamaño final del dataframe: {len(games)} registros ')

tamaño inicial del dataframe: 120445 registros
tamaño final del dataframe: 32135 registros 


In [225]:
# 2. Eliminar columnas que no utilizaremos: ``publisher``, ``url``, ``reviews_url``, ``price``, ``early_access``, ``developer``.
# Eliminar la columna ``tags`` previo a rellenar los datos faltantes de ``genres`` con sus valores. 
# ``app_name`` y ``title`` dicen lo mismo, dejaremos "app_name" que tiene menos datos faltantes y le cambiaremos el nombre a "title"

print(f"El tamaño inicial de games era {games.shape}")

games['genres'] = games['genres'].combine_first(games['tags']) # a cada valor faltante de genres le hara corresponder lo que figure en tags
games['title']= games['app_name']
games.drop(columns=['publisher', 'url', 'reviews_url', 'price', 'early_access', 'developer', 'tags', 'app_name'], inplace=True)

print("Columnas sin utilidad eliminadas")
print(f"Ahora, el tamaño final es: {games.shape}")


El tamaño inicial de games era (32135, 13)
Columnas sin utilidad eliminadas
Ahora, el tamaño final es: (32135, 5)


In [226]:
games.sample()

,genres,title,release_date,specs,id
28415,"[Simulation, Strategy]",To End All Wars,2014-08-29,"[Single-player, Multi-player, Co-op]",312360


In [227]:
# 3. Desanidar registros que poseen valores con tipo de dato LISTA: genres y specs.
games = games.explode(column=['genres'], ignore_index=True)
games = games.explode(column=['specs'], ignore_index=True)
games.shape

(408844, 5)

In [228]:
# 4. Eliminar registros duplicados.
games.drop_duplicates(inplace=True, ignore_index=True)
games.shape

(408840, 5)

In [229]:
# 5. Corregir el tipo de dato de cada columna.
## genres OK
## specs OK
## title OK
## release_date INCONSISTENTE
## id VALORES NAN. Borrarlos, no me sirven xq no puedo identificar sus reviews
fecha_relleno = pd.to_datetime('1900-01-01')
games['release_date'] = pd.to_datetime(games['release_date'], format='%Y-%m-%d', errors='coerce').fillna(fecha_relleno).dt.year
games.rename(columns={'release_date':'release_year','id':'item_id'}, inplace=True)
# Si no tiene ID no sirve, se dropea.
games.dropna(subset='item_id', inplace=True)
games['item_id']=games['item_id'].astype(int)
games.info()
games.shape

<class 'pandas.core.frame.DataFrame'>
Index: 408829 entries, 0 to 408839
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   genres        408631 non-null  object
 1   title         408825 non-null  object
 2   release_year  408829 non-null  int32 
 3   specs         407184 non-null  object
 4   item_id       408829 non-null  int32 
dtypes: int32(2), object(3)
memory usage: 15.6+ MB


(408829, 5)

In [230]:
# 6. Eliminar registros que poseen datos vacíos en columnas relevantes
item_id_sin_utilidad = games['item_id'][games['genres'].isna()].unique()
games.dropna(subset='genres', inplace=True, ignore_index=True)
games.shape

(408631, 5)

#### Para poder MERGEAR las tablas, los campos de las PK y FK deben ser el mismo tipo de dato. No puedo mergear un ``item_id`` string con un ``item_id`` integer

#### Reviews

In [232]:
## Reviews:
user_reviews = []
with gzip.open(ruta_user_reviews, 'rt', encoding="utf-8") as archivo_comprimido:
    for linea in archivo_comprimido:
        user_reviews.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
user_reviews_orig = pd.DataFrame(user_reviews)


In [262]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
user_reviews = user_reviews_orig.copy()

In [ ]:
user_reviews.head()

In [263]:
# Extraigo solo las columnas que voy a usar

reviews = user_reviews[['user_id','reviews']]

In [264]:
# Desanidamos reviews:

reviews_explode = reviews.explode('reviews', ignore_index=True) # separo en filas todos todos los json que estan listados en cada registro
reviews_desanidado = pd.json_normalize(reviews_explode['reviews']) # transformo a tabla cada uno de los json que tenemos en cada registro
reviews = pd.concat([reviews_explode,reviews_desanidado], axis=1).drop(columns=['reviews']) # concateno ambos dataframes y dropeo la columna reviews que está anidada

In [ ]:
reviews.head()

In [265]:
reviews.shape

(59333, 8)

Las transformaciones que realizaremos en este caso serán: 

1. Removeremos los registros en el que la mayoria de los campos son NaN.
2. Removeremos la columna ``funny``, no nos interesa saber si una opinion es divertida, mas bien nos interesa si da una buena, mala o neutral opinion del juego. Tampoco nos interesa la última edicion del review, entonces ``last_edited`` será eliminada.
3. Corrección de la fecha de publicación del review, cambio del tipo de dato y renombramiento de ```posted`` a ``posted_year``.
4. Remover los registros es los que ``item_id`` coincide con **item_id_sin_utilidad**. Corresponden a ids de juegos que ya fueron borrados de nuestra BBDD.
5. Cambio del tipo de dato de la columna ``recommend``.


In [266]:
reviews[reviews['funny'].isna()].shape # todos estos son registros practicamente nulos

(28, 8)

In [267]:
# 1. Removeremos los registros en el que la mayoria de los campos son NaN.
reviews.dropna(subset='funny', inplace=True, ignore_index=True)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   funny        59305 non-null  object
 2   posted       59305 non-null  object
 3   last_edited  59305 non-null  object
 4   item_id      59305 non-null  object
 5   helpful      59305 non-null  object
 6   recommend    59305 non-null  object
 7   review       59305 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


In [268]:
# 2. Removeremos la columna ``funny``, no nos interesa saber si una opinion es divertida,
# mas bien nos interesa si da una buena, mala o neutral opinion del juego.
reviews.drop(columns=['funny', 'last_edited'], inplace=True)
reviews.sample()

,user_id,posted,item_id,helpful,recommend,review
23292,76561198090341048,"Posted June 21, 2014.",267550,No ratings yet,True,10/10 your webs stick to buildings instead of ...


In [269]:
# 3. Corrección de la fecha de publicación del review, cambio del tipo de dato
a_eliminar = reviews['posted'].str.contains(r'Posted [A-Z][a-z]+ \d{1,2}\.')
reviews=reviews[~a_eliminar]
reviews.reset_index(drop=True, inplace=True)
def extraer_año(texto):
    return texto[-5:-1]
reviews['posted'] = reviews['posted'].apply(lambda x: extraer_año(x)).astype('int32')
reviews.rename(columns={'posted':'posted_year'}, inplace=True)


In [270]:
# 4. Remover los registros es los que ``item_id`` coincide con **item_id_sin_utilidad**. Corresponden a ids de juegos que ya fueron borrados de nuestra BBDD.
reviews['item_id'] = reviews['item_id'].astype('int32')
reviews = reviews[~reviews['item_id'].isin(item_id_sin_utilidad)].reset_index(drop=True)
# solo se borró un registro, pero bueno, cualquier limpieza por mas chica que sea es bienvenida. 

In [274]:
reviews.shape

(49185, 6)

In [278]:
reviews['recommend'] = reviews['recommend'].astype('bool') 

#### Items

In [279]:
## Items:
user_items = []
with gzip.open(ruta_user_items, 'rt', encoding="utf-8") as archivo_comprimido:
    for linea in archivo_comprimido:
        user_items.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
user_items_orig = pd.DataFrame(user_items)

In [310]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
user_items = user_items_orig.copy()

In [311]:
# Extraigo solo las columnas que voy a usar:
items = user_items[['user_id','items']]
items.head()

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [312]:
# Desanidamos items:

items_explode = items.explode('items', ignore_index=True) # separo en filas todos todos los json que estan listados en cada registro
items_desanidado = pd.json_normalize(items_explode['items']) # transformo a tabla cada uno de los json que tenemos en cada registro
items = pd.concat([items_explode,items_desanidado], axis=1).drop(columns=['items']) # concateno ambos dataframes y dropeo la columna items que está anidada

In [313]:
items.shape

(5170015, 5)

In [314]:
items.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0


Las transformaciones que realizaremos en este caso serán: 

1. Removeremos los registros en el que la mayoria de los campos son NaN.

In [315]:
items.shape
items.dropna(subset='item_id', inplace=True, ignore_index=True)
items.shape

(5153209, 5)

In [316]:
items[items['item_id'].isna()]

,user_id,item_id,item_name,playtime_forever,playtime_2weeks


#### Transformaciones generales

In [ ]:
games['title'].info()
games['app_name'].info()
# Son columnas similares. En este caso la columna que voy a dejar es app_name porque posee menor cantidad de datos vacios: 2.

In [ ]:
# Prueba para rellenar valores faltantes de genres
prueba = games[['genres','tags']].loc[88313:88320]
prueba['genres'] = prueba['genres'].combine_first(prueba['tags'])
prueba

In [ ]:
games['genres'] = games['genres'].combine_first(games['tags']) # a cada valor faltante de genres le hara corresponder lo que figure en tags
games.drop(columns=['publisher'])

In [ ]:
# Tratamiento de registros vacíos
print(f"El tamaño inicial del dataset games es: \n{games.shape}")
print(f"El tamaño inicial del dataset reviews es: \n{reviews.shape}")
print(f"El tamaño inicial del dataset items es: \n{items.shape}")

games.dropna(how='all', inplace=True)
reviews.dropna(how='all', inplace=True)
items.dropna(how='all', inplace=True)

print(f"Removiendo los registros completamente vacíos nos quedaran con un tamaño de \n games: {games.shape}")
print(f"Removiendo los registros completamente vacíos nos quedaran con un tamaño de \n reviews: {reviews.shape}")
print(f"Removiendo los registros completamente vacíos nos quedaran con un tamaño de \n items: {items.shape}")

In [ ]:
# Tratamiento de registros duplicados
print(f"El tamaño del dataset games previo a realizar limpieza de registros duplicados es: {games.shape}")
print(f"El tamaño del dataset reviews previo a realizar limpieza de registros duplicados es: {reviews.shape}")
print(f"El tamaño del dataset items previo a realizar limpieza de registros duplicados es: {items.shape}")

games.drop_duplicates(subset=['publisher', 'app_name', 'title', 'url', 'release_date', 'reviews_url', 'price', 'early_access', 'id','developer'])
reviews.drop_duplicates()
items.drop_duplicates()

print(f"El tamaño del dataset games luego de realizar limpieza de registros duplicados es: {games.shape}")
print(f"El tamaño del dataset reviews luego de realizar limpieza de registros duplicados es: {reviews.shape}")
print(f"El tamaño del dataset items luego de realizar limpieza de registros duplicados es: {items.shape}")

#### 3. Carga o disponibilización de datos

#### 3.1 PlayTimeGenre:  Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [ ]:
games.tail()

In [ ]:
reviews.head()

In [ ]:
items.head()

#### 3.2 UsersForGenre: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [ ]:
items[['user_id','item_id','playtime_forever']].head()